In [2]:
# imports
import pandas as p                              # it's pandas
from sklearn.preprocessing import MinMaxScaler  # for features scaling
import Constants as c                           # constants
import Functions as f                           # plotting functions
import numpy as np
import pickle
import folium
from folium.plugins import HeatMap
import webbrowser as wb
import hdbscan as hdbs

In [3]:
# Import dataframe from "out" folder
with open('../out/TREAD_out.pkl', 'rb') as file:
    df, routes_list, name_of_route = pickle.load(file)

test_route = routes_list[1]

In [4]:
# some info about the dataframe

test_route.reset_index(inplace=True, drop=True)
if 'index' in test_route.columns:
    print("Column \"Index\" removed")
    # the above instruction sometimes adds a new column called 'index', which is subsequently removed
    test_route.drop(['index'], axis=1, inplace=True)
display(test_route)
#display(df)
# current columns (aka features) of the dataset
print(df.columns)
display(df.info())

,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,VesselType,Status,...,Draft,IsClassA,Avg_Speed,EstimatedStatus,IsInWP,IsInSignificantWP,Route,IsInRoute,group,route_id
0,338143427,2019-01-08 00:32:22,20.7358,-156.8890,6.9,116.0,511.0,KOA,36,99,...,NaN,False,0.53,sailing,WP_118,<NA>,WP_118 - WP_164,True,1,4937
1,338143427,2019-01-08 00:33:52,20.7364,-156.8862,6.3,74.8,511.0,KOA,36,99,...,NaN,False,6.46,sailing,WP_118,<NA>,WP_118 - WP_164,True,0,4937
2,338143427,2019-01-08 00:36:21,20.7376,-156.8818,6.0,76.3,511.0,KOA,36,99,...,NaN,False,6.22,sailing,WP_118,<NA>,WP_118 - WP_164,True,0,4937
3,338143427,2019-01-08 00:37:22,20.7381,-156.8800,6.1,74.9,511.0,KOA,36,99,...,NaN,False,6.23,sailing,None,<NA>,WP_118 - WP_164,True,0,4937
4,338143427,2019-01-08 00:38:54,20.7388,-156.8774,6.1,64.5,511.0,KOA,36,99,...,NaN,False,5.95,sailing,None,<NA>,WP_118 - WP_164,True,0,4937
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,338143427,2019-01-08 03:13:51,20.8544,-156.6858,5.8,28.5,511.0,KOA,36,99,...,NaN,False,6.01,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,4937
101,338143427,2019-01-08 03:14:53,20.8558,-156.6850,6.0,27.2,511.0,KOA,36,99,...,NaN,False,5.54,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,4937
102,338143427,2019-01-08 03:16:22,20.8581,-156.6843,5.8,14.4,511.0,KOA,36,99,...,NaN,False,5.81,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,4937
103,338143427,2019-01-08 03:17:52,20.8606,-156.6840,6.2,13.3,511.0,KOA,36,99,...,NaN,False,6.05,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,4937


Index(['MMSI', 'BaseDateTime', 'LAT', 'LON', 'SOG', 'COG', 'Heading',
       'VesselName', 'VesselType', 'Status', 'Length', 'Width', 'Draft',
       'IsClassA', 'Avg_Speed', 'EstimatedStatus', 'IsInWP',
       'IsInSignificantWP', 'Route', 'IsInRoute', 'group', 'route_id'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
Index: 9525534 entries, 0 to 40
Data columns (total 22 columns):
 #   Column             Dtype        
---  ------             -----        
 0   MMSI               int32        
 1   BaseDateTime       datetime64[s]
 2   LAT                float64      
 3   LON                float64      
 4   SOG                float64      
 5   COG                float64      
 6   Heading            float64      
 7   VesselName         string       
 8   VesselType         int16        
 9   Status             int8         
 10  Length             int16        
 11  Width              int16        
 12  Draft              float32      
 13  IsClassA           bool 

None

#### General Approach
1. Normalization (some features have different scales) (to be decided, probably not)
2. Features selection
3. Algorithm

### Normalization

In [5]:
test_route.isnull().sum()

MMSI                   0
BaseDateTime           0
LAT                    0
LON                    0
SOG                    0
COG                    0
Heading                0
VesselName             0
VesselType             0
Status                 0
Length                 0
Width                  0
Draft                105
IsClassA               0
Avg_Speed              0
EstimatedStatus        0
IsInWP                76
IsInSignificantWP    105
Route                  0
IsInRoute              0
group                  0
route_id               0
dtype: int64

In [6]:
# scaling, to be decided
'''
df = df.sort_values(by=['MMSI','BaseDateTime'])
#selected_cols = ['LAT','LON','SOG','COG','Heading','Length','Width','Avg_Speed']
selected_cols = ['LAT','LON','SOG','COG','Heading','Avg_Speed']

scaler = MinMaxScaler()
df[cols_to_be_scaled] = scaler.fit_transform(df[cols_to_be_scaled])

display(df.head(3))

# create a list of routes from the scaled dataframe df


only_route_points = df[(df['IsInRoute'] == True) & (df['Route'] == name_of_route)]

list_of_routes = [route[selected_cols]   for _, route in only_route_points.groupby('route_id')]


display(only_route_points.head())

display(list_of_routes[0])
'''

"\ndf = df.sort_values(by=['MMSI','BaseDateTime'])\n#selected_cols = ['LAT','LON','SOG','COG','Heading','Length','Width','Avg_Speed']\nselected_cols = ['LAT','LON','SOG','COG','Heading','Avg_Speed']\n\nscaler = MinMaxScaler()\ndf[cols_to_be_scaled] = scaler.fit_transform(df[cols_to_be_scaled])\n\ndisplay(df.head(3))\n\n# create a list of routes from the scaled dataframe df\n\n\nonly_route_points = df[(df['IsInRoute'] == True) & (df['Route'] == name_of_route)]\n\nlist_of_routes = [route[selected_cols]   for _, route in only_route_points.groupby('route_id')]\n\n\ndisplay(only_route_points.head())\n\ndisplay(list_of_routes[0])\n"

### Algorithm

In [7]:
'''
testDF = p.concat(routes_list, ignore_index=True, axis=0)

# create Heatmap using Folium
map = folium.Map(location=[testDF.LAT.mean(), testDF.LON.mean()], zoom_start=12)

chosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']
heat_data = testDF[['LAT', 'LON']].values.tolist()
heat_data_DF = testDF[chosen_columns]

HeatMap(heat_data, radius=14).add_to(map)

map.save('test.html')
wb.open('file://' + os.path.realpath('test.html'))
'''


"\ntestDF = p.concat(routes_list, ignore_index=True, axis=0)\n\n# create Heatmap using Folium\nmap = folium.Map(location=[testDF.LAT.mean(), testDF.LON.mean()], zoom_start=12)\n\nchosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']\nheat_data = testDF[['LAT', 'LON']].values.tolist()\nheat_data_DF = testDF[chosen_columns]\n\nHeatMap(heat_data, radius=14).add_to(map)\n\nmap.save('test.html')\nwb.open('file://' + os.path.realpath('test.html'))\n"

In [11]:
#another test

testDF = p.concat(routes_list, ignore_index=True, axis=0)
chosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading']
#chosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']
chosen_data = testDF[chosen_columns]

from sklearn.mixture import GaussianMixture

lowest_bic = np.inf
best_gmm = None
n_clusters_range = range(1,11)


for cluster_num in n_clusters_range:
    gmm = GaussianMixture(n_components= cluster_num, random_state = 0)
    gmm.fit(chosen_data)
    bic = gmm.bic(chosen_data)

    if bic < lowest_bic:
        lowest_bic = bic
        best_gmm = gmm

testDF['cluster'] = best_gmm.predict(chosen_data)

map = folium.Map(location=[testDF.LAT.mean(), testDF.LON.mean()], zoom_start=12)

colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen']

#plot
f.route_plot(inputDF=testDF, color_criteria='cluster', hover_data=chosen_columns, title=f'Route {name_of_route}')
# TODO fix this function below 
#f.route_clusters_plot(testDF)

In [12]:
# Select biggest clusters until 80% of points is reached
testDF['cluster'].value_counts()
total_points = testDF['cluster'].size
eighty_percent = int(total_points * 0.8)
display(total_points, eighty_percent)

groupedDF = testDF.groupby(by='cluster').size().reset_index(name='count',).sort_values(by='count', ascending=False)
groupedDF.reset_index(drop=True, inplace=True)
display(groupedDF)

partial_sum = 0
for index, row in groupedDF.iterrows():
    temp_value = row['count']
    if partial_sum + temp_value > eighty_percent:
        break
    else:
        partial_sum = partial_sum + temp_value

selected_clusters = groupedDF.iloc[:index+1, 0].to_list()

display(selected_clusters)

f.route_plot(testDF[testDF['cluster'].isin(selected_clusters)], color_criteria='cluster')
f.route_plot(testDF[testDF['cluster'].isin(set(groupedDF['cluster'].to_list()) - set(selected_clusters))], color_criteria='cluster')

18511

14808

,cluster,count
0,6,8837
1,9,4019
2,0,1419
3,3,1409
4,5,1296
5,1,509
6,7,435
7,2,261
8,8,209
9,4,117


[6, 9, 0, 3]

##### Test with hdbscan

In [10]:
testDF = p.concat(routes_list, ignore_index=True, axis=0)
chosen_columns = ['LAT', 'LON', 'Avg_Speed', 'COG', 'Heading', 'IsClassA']
chosen_data = testDF[chosen_columns]

# inizitalize h-dbscan object with parameters. TODO: test multiple parameters combinations
h_dbscan = hdbs.HDBSCAN(min_cluster_size=30,
                        cluster_selection_epsilon=0.01,
                        max_cluster_size=10_000)

h_labels = h_dbscan.fit_predict(
                    X=testDF[['LAT','LON']]
                    )

print(str(h_dbscan) + " len: " + str(len(h_labels)) + ", num of clusters: " + str(max(h_labels)))

# Add labels to the dataframe
testDF['Label'] = h_labels

display(testDF)

# Show the result of h-dbscan on a map

clusteredPointsDF = testDF[(testDF['Label'] != -1)].copy()
outliersDF = testDF[testDF['Label'] == -1].copy()

clusteredPointsDF = clusteredPointsDF.groupby(by=['LAT', 'LON', 'Label']).size().reset_index(name='Weight')

# plot points and clusters
f.points_plotter(clustered=clusteredPointsDF,
               not_clustered=outliersDF,
               model=h_dbscan)


HDBSCAN(cluster_selection_epsilon=0.01, max_cluster_size=10000,
        min_cluster_size=30) len: 18511, num of clusters: 2


,MMSI,BaseDateTime,LAT,LON,SOG,COG,Heading,VesselName,VesselType,Status,...,IsClassA,Avg_Speed,EstimatedStatus,IsInWP,IsInSignificantWP,Route,IsInRoute,group,route_id,Label
0,338143427,2019-01-02 19:53:11,20.7387,-156.8818,6.9,116.3,511.0,KOA,36,99,...,False,6.38,sailing,WP_118,<NA>,WP_118 - WP_164,True,1,4929,1
1,338143427,2019-01-02 19:58:08,20.7386,-156.8725,6.3,87.2,511.0,KOA,36,99,...,False,6.34,sailing,None,<NA>,WP_118 - WP_164,True,0,4929,2
2,338143427,2019-01-02 19:59:10,20.7391,-156.8707,6.7,64.2,511.0,KOA,36,99,...,False,6.13,sailing,None,<NA>,WP_118 - WP_164,True,0,4929,2
3,338143427,2019-01-02 20:00:10,20.7401,-156.8690,6.8,51.2,511.0,KOA,36,99,...,False,6.77,sailing,None,<NA>,WP_118 - WP_164,True,0,4929,2
4,338143427,2019-01-02 20:02:08,20.7418,-156.8659,3.2,73.6,511.0,KOA,36,99,...,False,6.16,sailing,None,<NA>,WP_118 - WP_164,True,0,4929,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18506,367770790,2019-05-22 01:02:40,20.8779,-156.6924,8.8,175.9,511.0,LOKAHI,36,99,...,False,1.70,return,WP_164,<NA>,WP_118 - WP_164,True,0,54491,-1
18507,367770790,2019-05-22 01:09:40,20.8716,-156.6824,1.7,89.4,511.0,LOKAHI,36,99,...,False,5.81,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,54491,2
18508,367770790,2019-05-22 01:11:40,20.8718,-156.6828,5.3,295.4,511.0,LOKAHI,36,99,...,False,0.76,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,54491,2
18509,367770790,2019-05-22 01:16:40,20.8707,-156.6846,6.1,184.6,511.0,LOKAHI,36,99,...,False,1.45,sailing,WP_164,<NA>,WP_118 - WP_164,True,0,54491,2
